In [ ]:
import pandas as pd
import requests
from io import StringIO 

from datetime import datetime

import os


if not os.path.exists('/tmp/upload/'):
    os.mkdir('/tmp/upload/')


In [ ]:
def request_to_string(url):
    r = requests.get(url)
    StringData = StringIO(r.text)
    return StringData

def process_dataframe(url,is_US,filename):
    StringData = request_to_string(url)

    if is_US:
        df_raw, df_convenient = process_US(StringData)
    else:
        df_raw, df_convenient = process_global(StringData)
     
    df_raw.to_csv(f'/tmp/upload/RAW_{filename}.csv')
    
    df_convenient.drop(['1/22/20'],inplace=True)
    df_convenient.to_csv(f'/tmp/upload/CONVENIENT_{filename}.csv')

    return df_raw, df_convenient


def process_global(StringData):
    df_raw = pd.read_csv(StringData,index_col=[1,0]) 
    df_convenient = df_raw.drop(['Lat','Long'],axis=1).T.diff(1)
    
    df_metadata = df_raw[['Lat','Long']] 
    df_metadata.to_csv(f'/tmp/upload/CONVENIENT_global_metadata.csv')  
    
    return df_raw, df_convenient


def process_US(StringData):
    df_raw = pd.read_csv(StringData,index_col=[6,5])
    df_convenient = df_raw.drop(['UID','iso2','iso3','code3','FIPS','Country_Region','Lat','Long_','Combined_Key','Population'],axis=1,errors='ignore').T.diff(1)
    
    if 'Population' in df_raw.columns:
        df_us_metadata = df_raw[['Population','Lat','Long_']]
        df_us_metadata.columns = ['Population','Lat','Long']   
        df_us_metadata.to_csv(f'/tmp/upload/CONVENIENT_us_metadata.csv')
        
    return df_raw, df_convenient



In [ ]:
df_confirmed_global_raw, df_confirmed_global_convenient = process_dataframe('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true',False,'global_confirmed_cases')
df_deaths_global_raw, df_deaths_global_convenient = process_dataframe('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv?raw=true', False,'global_deaths')
df_confirmed_us_raw, df_confirmed_us_convenient = process_dataframe('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv?raw=true',True,'us_confirmed_cases')
df_deaths_us_raw, df_deaths_us_convenient = process_dataframe('https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv?raw=true',True,'us_deaths')
